<a href="https://colab.research.google.com/github/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG2/WilliamSteveRodriguezVillamizar_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG1- Actividad Guiada 2<br>
Nombre y Apellidos:   <br>
Colab: https://colab.research.google.com/github/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG2/WilliamSteveRodriguezVillamizar_AG2.ipynb<br>

Url:: https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG2/WilliamSteveRodriguezVillamizar_AG2.ipynb<br>


In [0]:
#Decorador de Ricardo Lebron
from functools import wraps
from time import time

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time() #tomo la hora antes de ejecutar la funcion
        salida = f(*args, **kwargs)
        t_final = time() #tomo la hora despues de ejecutar la funcion
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [0]:
#Programación dinámica
#C(i,j) = min {T(i,j) , C(i,k)+T(k,j) para todo i<k<=j } 
def PRECIOS(TARIFAS):
  N = len(TARIFAS[0])
  
  PRECIOS = [[9999]*N for i in [9999]*N]
  RUTA = [[""]*N for i in [""*N]]
  
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
          MIN = min(MIN, PRECIOS[j][k] + TARIFAS[k][j])
          RUTA[i][j] = k
        PRECIOS[i][j] = MIN
  return PRECIOS, RUTA



In [5]:
#Fuerza bruta
def valor(S, COSTES):
  VALOR = 0
  for i in range(len (S)):
    VALOR += COSTES[S[i]][i]
  return VALOR

@calcular_tiempo
def Fuerza_bruta(COSTES):
  mejor_valor = 10e10
  mejor_solucion = ()
  
  for s in list(itertools.permutations(range(len(COSTES)))):
    valor_tmp = valor(s, COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s
  print("La mejor solución es: ", mejor_solucion, " con valor: ", mejor_valor)
    
COSTES = [[11,12,18,400], [14,15,13,22],[11,17,19,23],[17,14,20,28]]  
valor((0,1,2,3), COSTES)

73

In [0]:
#coste inferior solución parcial
def CI(S, COSTES):
  VALOR = 0
  #valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]
  
  #Estimación
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += min([COSTES[j][i] for j in range(len(S), len(COSTES))])
      
  return VALOR

In [0]:
#Funcion para ramificar
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO+(i,)})
  return HIJOS

In [0]:
#Ramificación y poda

@calcular_tiempo
def ramificacion_y_poda(COSTES):
  DIMENSION = len(COSTES)
  MEJOR_SOLUCION = tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION, COSTES)
  
  NODOS = []
  NODOS.append({'s':(),'ci':CI((), COSTES)})
  iteracion = 0
  
  while(len(NODOS)>0):
    iteracion += 1
    
    nodo_prometedor = [min(NODOS, key=lambda x:x['ci'])][0]['s']
    
    HIJOS = [ {'s':x['s'], 'ci':CI[x['s'], COSTES]} for x in crear_hijos(nodo_prometedor, DIMENSION)   ]
    
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) >0:
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
        
    #poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]
    
    #añadimos los hijos
    NODOS.extend(HIJOS)
    
    #eliminamos el nodo ramificado
    NODOS = [x for x in NODOS if x['s'] != nodo_prometedor]
  print("La solución final es: ",MEJOR_SOLUCION , " en ", iteracion, " iteraciones ", "para dimensión: ", DIMENSION )
      
    
    
    
    